In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022306442260742188
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 10.861437285314251
200000 21.204537002508772
300000 25.181403751692773
400000 25.168059849533435
500000 26.44872953104338
600000 26.363940471580964
700000 25.657056334436255
800000 24.60461250250936
900000 25.79870356899474
1000000 12.662369844013378
1100000 22.503826109546967
1200000 24.80173312407255
1300000 23.820607764596176
1400000 26.323350009096615
1500000 26.123447069323404
1600000 24.9876335594399
1700000 24.054037840770125
1800000 25.015418233771367
1900000 25.62573339117774
2000000 7.853817370338146
2100000 28.20359969275385
2200000 29.900861750094137
2300000 30.21240705988019
2400000 32.062308999784264
2500000 28.96333531093163
2600000 28.263536197371693
2700000 29.253905738992067
2800000 28.185218741997677
2900000 26.2128784219682
3000000 12

29400000 24.997961540617876
29500000 23.99836106931188
29600000 10.61465023022778
29700000 22.689639622135562
29800000 23.344171599543966
29900000 22.7938367072774
30000000 21.571662072984175
30100000 21.13731218651406
30200000 20.342259359428944
30300000 9.685686829149818
30400000 19.398567490323337
30500000 20.103964165807277
30600000 21.73242178164792
30700000 21.542386376757975
30800000 22.8538807051977
30900000 22.626737058425224
31000000 18.251947779432715
31100000 16.76320293743025
31200000 22.743565925697563
31300000 24.031048313236926
31400000 23.36160291932875
31500000 18.770606816163465
31600000 15.377439233063535
31700000 23.34586126280166
31800000 23.757908080794756
31900000 23.91050345342977
32000000 23.510147531377157
32100000 21.607546525596977
32200000 19.283966204586648
32300000 10.308479314242039
32400000 22.22341901917992
32500000 21.18961630445648
32600000 22.224669099657895
32700000 22.398406679198107
32800000 24.062070081814685
32900000 22.75211608086021
33000000

59100000 10.089610831985757
59200000 17.53754961974024
59300000 19.679375043376016
59400000 18.965544755853667
59500000 19.065188196003902
59600000 8.16384738288599
59700000 18.577963498263717
59800000 19.37617450241798
59900000 18.255720495616433
60000000 19.629786347532722
60100000 9.731641528578622
60200000 19.26743310881345
60300000 20.483573322560016
60400000 20.849501621955266
60500000 20.911931190287323
60600000 22.5112854684406
60700000 20.893683923058948
60800000 20.336734811670468
60900000 20.41764361122338
61000000 20.7917753291228
61100000 8.04832970299004
61200000 18.728672554275022
61300000 17.916767835938334
61400000 18.289925256180844
61500000 16.253647583423437
61600000 9.276632534842863
61700000 18.876085971355796
61800000 18.504352878726397
61900000 18.065764341884755
62000000 14.972171141305108
62100000 14.38438072519671
62200000 20.52356084550192
62300000 20.83225240578761
62400000 19.579312147771482
62500000 20.638788183732977
62600000 20.940429384475156
62700000 

88900000 6.254502971986988
89000000 17.183406185550446
89100000 16.054571838129103
89200000 9.4832424528856
89300000 16.248786800135175
89400000 15.854447518980683
89500000 16.644326242301016
89600000 16.374692232340216
89700000 17.292573144281125
89800000 17.28355021078818
89900000 18.420424029130007
90000000 17.75815781385423
90100000 17.009785455805392
90200000 17.035788375499656
90300000 16.783487408133297
90400000 16.52571463018758
90500000 15.675976057275278
90600000 8.380139513251637
90700000 15.776772795071494
90800000 14.08562534300915
90900000 8.669821180925075
91000000 15.98738009996399
91100000 8.685368838893238
91200000 15.794183595977902
91300000 16.43171029610145
91400000 16.165521656605623
91500000 16.38917987694192
91600000 17.0345755947172
91700000 18.35917928370682
91800000 16.797401231629745
91900000 18.358091486991587
92000000 18.02965338831977
92100000 17.81842132624433
92200000 16.134303026520865
92300000 16.366541489765208
92400000 15.726385946067078
92500000 16

118000000 1.6051336832830065
118100000 1.5311994248258014
118200000 1.6337609346658646
118300000 1.6293977799639834
118400000 1.6905696966433088
118500000 1.5748892124701204
118600000 1.5353175907650674
118700000 1.5561785556006271
118800000 1.5076420953582983
118900000 1.5077093331157905
119000000 1.5128236615491988
119100000 1.5329114881974877
119200000 1.525528382957832
119300000 1.5380411130648362
119400000 1.5307761212531725
119500000 1.5065297805249684
119600000 1.7518348403358839
119700000 1.524682223328801
119800000 1.5109844634633978
119900000 2.1236602536348808
120000000 1.6397138116743564
120100000 1.5629653948116904
120200000 1.5365064680943357
120300000 1.701472165001446
120400000 1.6025530842761144
120500000 1.5717714289612539
120600000 1.6177325556506574
120700000 1.7520183726069127
120800000 1.5542927001864912
120900000 1.5867164076995193
121000000 1.6735812686638878
121100000 1.5330008817402199
121200000 1.6327235651600718
121300000 1.5630327811668627
121400000 1.58976

...

204400000 7.779982342893742
204500000 8.770820818421994
204600000 8.351417361474256
204700000 8.595999296888708
204800000 3.8435075413882815
204900000 8.933802881137487
205000000 8.573696270657853
205100000 8.8094932663982
205200000 8.38683625375958
205300000 8.330836944057628
205400000 5.027727220598413
205500000 7.651178112982095
205600000 8.092306518966579
205700000 7.901993359550207
205800000 8.024852340051941
205900000 7.39722576552079
206000000 7.494779363353022
206100000 2.917540472922969
206200000 7.679059510955261
206300000 7.462875715226213
206400000 7.8273158645680185
206500000 7.998060665552283
206600000 8.247857807636134
206700000 7.970096761250383
206800000 4.342602148787325
206900000 8.360960653299554
207000000 8.365237662407491
207100000 8.796454570928868
207200000 8.49403975018477
207300000 7.364368546979152
207400000 5.516624026935395
207500000 8.601670936837184
207600000 8.231624053568227
207700000 8.204484497260005
207800000 8.067529256609353
207900000 7.63721569281

233700000 6.258238246468683
233800000 6.298351780549344
233900000 6.443519145110962
234000000 6.644536018315207
234100000 6.4702741724028465
234200000 6.578596353199004
234300000 4.184993272529608
234400000 5.495932132061961
234500000 3.2448059546434775
234600000 6.502705541594185
234700000 6.357773562821803
234800000 6.5873298728791285
234900000 6.527294817130895
235000000 6.14961658574629
235100000 6.205011446104385
235200000 6.4036374532552065
235300000 6.344617910569438
235400000 2.2872476347866324
235500000 6.560494392783985
235600000 6.467103986204784
235700000 6.2901795147821655
235800000 6.340785309219555
235900000 6.716130755888217
236000000 6.545377487858377
236100000 6.444771407400853
236200000 6.484692139253331
236300000 2.7652733937636444
236400000 4.091107046966279
236500000 5.649342159937426
236600000 6.319200516028183
236700000 6.374753430679161
236800000 6.485875744360243
236900000 6.162931428390627
237000000 5.945507889096857
237100000 6.045149670367653
237200000 6.26

262300000 1.8313760933607843
262400000 1.743578005229648
262500000 2.2448566416119338
262600000 1.1336909257752876
262700000 0.48775116521942424
262800000 0.48899628147985735
262900000 1.2045821347284131
263000000 2.475659486012533
263100000 1.76621924567973
263200000 1.949483054294284
263300000 1.957552222978319
263400000 1.791044838234943
263500000 1.9639705612654024
263600000 1.9108162289958133
263700000 2.249336734859943
263800000 2.0645595759382283
263900000 2.033806255818043
264000000 1.8459088577703
264100000 1.7434532270698977
264200000 1.7201211613221512
264300000 1.907545208467418
264400000 1.9695511850652063
264500000 1.5153825037039612
264600000 0.7870178544161817
264700000 0.4701890949108435
264800000 0.659980126559584
264900000 1.5566644049143852
265000000 2.1327930290777597
265100000 1.866684862759475
265200000 1.6929263219166737
265300000 1.742653387092816
265400000 1.8390677692854696
265500000 1.9365990950206338
265600000 1.9225252780061877
265700000 2.144247428440081


290800000 1.0784988739773584
290900000 1.0698882940268597
291000000 1.1239623391421079
291100000 1.0956463957305067
291200000 1.1349988652260667
291300000 0.9899810412184497
291400000 0.9745239530281735
291500000 1.0773807778234898
291600000 0.8820285731289188
291700000 0.7324100575086215
291800000 1.0832769711714922
291900000 0.5394260036408305
292000000 0.23769130635376773
292100000 0.2172326434663415
292200000 0.5711710325990756
292300000 0.9662299929833565
292400000 0.7694175125561621
292500000 0.9907749870693624
292600000 0.9706752605500122
292700000 0.8702054415685614
292800000 1.0050175408929527
292900000 0.9911940424486426
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 1.0246710844566054
293100000 0.8113334264989734
293200000 0.7771529803219749
293300000 0.8452823831486344
293400000 0.751985209099255
293500000 0.7175832000064771
293600000 0.6428954761455503
293700000 0.6545409041301489
293800000 0.7134981637292948
293900000 0.5949983914143966
29

best so far: 0
type: [1, 1, 25, 5, 1] 125
cases of this type: 78125
317200000 0.021605902155929144
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.02088519966865182
317400000 0.06721284069544475
317500000 0.042273968564326234
317600000 0.035968994473146734
317700000 0.05175521753717264
317800000 0.03319255915555623
317900000 0.0424121538193643
318000000 0.03577812888255715
318100000 0.02854853796825608
318200000 0.014846438954506318
318300000 0.01414921808048288
318400000 0.013452673261510662
318500000 0.026625406223381894
318600000 0.03477612596295675
318700000 0.025307207263449826
318800000 0.01823682532040543
318900000 0.017938662314802407
319000000 0.02624641314402421
319100000 0.025428579041812813
319200000 0.017517578324125208
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.009259999640549554
319400000 0.0074877130484177
319500000 0.00529121223286